In [4]:
import pumas
from pumas.desirability import desirability_catalogue

After the necessary imports, we can check the version of the `mpstk` package. 

In [2]:
print(pumas.__version__)

0.0.0


## Desirability Functions

Le's check the available desirability functions in the catalogue.
The output is a list of strings, each string is the name of a desirability function, that can be used to create an instance of the corresponding class.

In [1]:
desirability_catalogue.list_items()

NameError: name 'desirability_catalogue' is not defined

We can use the catalogue to create an instance of a desirability function.
The catalogue returns a class, and we can create an instance of this class by calling it.
This approach allows us to create an instance of a desirability function without importing the class directly.

In [15]:
desirability_class = desirability_catalogue.get("sigmoid")
desirability = desirability_class()

If we want to see the documentation of the class, we can use the `help` function.
This function will display the docstring of the class, which contains information about the class and its methods.

In [6]:
help(desirability)

Help on SigmoidDesirability in module mpstk.desirability.desirability object:

class SigmoidDesirability(BaseDesirability)
 |  The class :class:`SigmoidDesirability<mpstk.desirability.desirability.SigmoidDesirability>` implements a sigmoid desirability function.
 |  
 |  It implements the definition of input and coefficient parameters.
 |  
 |  This class wraps the sigmoid utility function and implements the interface to use it, identifying
 |      * input parameters: 'x'
 |      * coefficient parameters: 'low', 'high', 'k', 'base' and 'shift'.
 |  
 |  The sigmoid function has a flexible curve that is controlled by its parameters
 |  and can be used to model scenarios where desirability transitions smoothly
 |  between low and high values.
 |  
 |  Upon initialization, the attributes of some coefficient parameters are set to
 |  reasonable defaults, which can be changed by the user.
 |  
 |  
 |  Usage Example:
 |  
 |  >>> from mpstk.desirability.desirability import SigmoidDesirabili

Before using the desirability function, we need to set the parameters of the function.
We obtain a list of the parameters that the function needs by calling the `get_coefficient_parameters_values` method.  
The output is a dictionary, where the keys are the names of the parameters, and the values are the default values of the parameters.  
Some of the parameters are optional, and they have a default value.   
Some parameters have a None value, which means that the user needs to set the value of the parameter.  

In [16]:
print(desirability.get_coefficient_parameters_values())

{'low': None, 'high': None, 'k': 0.5, 'base': 10.0, 'shift': 0.0}


We can now set the values of the parameters of the desirability function and ensure that the values are set correctly by calling the `get_coefficient_parameters_values` method.


In [17]:
desirability.set_coefficient_parameters_values(
    {"low": 0.0, "high": 1.0, "k": 0.1, "shift": 0.0, "base": 10.0}
)

print(desirability.get_coefficient_parameters_values())

{'low': 0.0, 'high': 1.0, 'k': 0.1, 'base': 10.0, 'shift': 0.0}


The desirability function is now ready to be used.

In [18]:
result = desirability.compute_score(x=0.5)
print(result)

0.5


In [19]:
pm = desirability.parameters_map
print(pm)

{'x': FloatParameter(name='x', default=None, min=None, max=None), 'low': FloatParameter(name='low', default=None, min=None, max=None), 'high': FloatParameter(name='high', default=None, min=None, max=None), 'k': FloatParameter(name='k', default=0.5, min=-1.0, max=1.0), 'shift': FloatParameter(name='shift', default=0.0, min=0.0, max=1.0), 'base': FloatParameter(name='base', default=10.0, min=0.0, max=10.0)}
